## Distinguishing between left and right sides

Based on the page: https://towardsdatascience.com/i-built-a-brain-computer-interface-to-play-space-invaders-using-thoughts-23980cb4faf7 

### Data collection

Replace with our method of integrating to the Python API

In [ ]:
#REPLACE WITH PYTHON API COLLECTION METHOD!
"""
from pylsl import StreamInlet, resolve_stream
import pandas as pd

# initialize the streaming layer
finished = False
streams = resolve_stream()
inlet = StreamInlet(streams[0])

# initialize the colomns of your data and your dictionary to capture the data.
columns=['Time','FZ', 'C3', 'CZ', 'C4', 'PZ', 'PO7', 'OZ', 'PO8','AccX','AccY','AccZ',
'Gyro1','Gyro2','Gyro3', 'Battery','Counter','Validation']
data_dict = dict((k, []) for k in columns)

while not finished:
   # get the streamed data. Columns of sample are equal to the columns variable, only the first element being timestamp
   # concatenate timestamp and data in 1 list
   data, timestamp = inlet.pull_sample()
   all_data = [timestamp] + data
   
   # updating data dictionary with newly transmitted samples   
   i = 0
   for key in list(data.keys()):
      data_dict[key].append(all_data[i])
      i = i + 1
   
   # data is collected at 250 Hz. Let's stop data collection after 60 seconds. Meaning we stop when we collected 250*60 samples.
   if len(data_dict['Time']) >= 250*60:
      finished = True

# lastly, we can save our data to a CSV format.
data_df = pd.DataFrame.from_dict(data_dict)
data_df.to_csv('EEGdata.csv', index = False) 
"""

### Pre-processing:
 notch filter, common average referencing & artifact detection

In [ ]:
from scipy import signal, stats
import pandas as pd
import numpy as np


# Having our EEG data in a Pandas DataFrame as (timepoints, channels)

for curr_segment in segments:    
    # 1. notch filter
    b_notch, a_notch = signal.iirnotch(50, 30, sampling_frequency)
    for column in curr_segment.columns:
        curr_segment.loc[:,column] = signal.filtfilt(b_notch, a_notch, curr_segment.loc[:,column])
    
    # for next steps data should be in (channels, timepoints format)
    curr_segment = curr_segment.T

    # 2 OUTLIER DETECTION
    for i, j in curr_segment.iterrows():
        if stats.kurtosis(j) > 4*np.std(j) or (abs(j - np.mean(j)) > 125).any():
            if stats.kurtosis(j) > 4*np.std(j):
                print('due to kurtosis')
            outlier +=1

    # 3 APPLY COMMON AVERAGE REFERENCE (CAR)   
    if  'deep' in pipeline_type: 
        curr_segment -= curr_segment.mean()

Frequency filtering

In [ ]:
# MAKE SURE DATA IS IN A VARIABLE NAMED 'data'

In [ ]:
from scipy import signal 
import pandas as pd


# We want to apply a bandpass between 4 and 40 Hz
cutoffs = [4, 40]
# the sample frequency of our device is 250Hz
fs = 250

# we have to device the order of the filter.
# the order determines the sharpness of the filter, i.e., around the # cutoff, do we allow some of the nearby frequency through (lower 
# order) or do we want to have a sharp cutoff (high order)? A higher 
# order introduces a bigger effect on the signal when initializing, 
# thereby destroying the original signal at that part. Note that 
# later we apply forward and backward filtering using 
# signal.filtfilt, thereby essentially doubling the order we 
# determine below.
order = 2

# initialize the filter coefficients
b, a = signal.butter(order, [cuttoff[0]/(fs/2), cuttoff[1]/(fs/2)], "bandpass")

# then apply the filter for each electrode channel:
for column in data.columns:
   data.loc[:,column] = signal.filtfilt(b, a, data.loc[:,column])

### Prediction

ML time

In [ ]:
# added by Ari
#TODO: split precollected, labelled data into X_train and y_train

#e.g. 
#from sklearn.model_selection import train_test_split
#X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.pipeline import Pipeline
from mne.decoding import CSP
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace

In [ ]:
# initialization of the pipeline, fit, and predict. 
# 2 possible methods

# Method 1: CSP + LDA
csp = Pipeline(steps=[('csp', CSP()),('lda', LDA())])
csp.fit(X_train, y_train)
preds = csp.predict(X_val)

# Method 2: Covariance + Tangent Space + Random Forest
rg = Pipeline(steps=[('cov', Covariances("oas")),('tg', TangentSpace(metric="riemann")),('rf', RFC())])
rg.fit(X_train, y_train)
preds = rg.predict(X_val)

### Using predictions

# choose one of the 2 preds above, whichever works best to give preds

prediction = preds[0]
direction = ''
while playing_game:
  # collect data in segments of 2 seconds (using PyLSL)
  # pre_process (notch, artifact detection, CAR) 
  # feature_extraction (frequency and spatial filtering), 
  # get_prediction (from trained model)
  # 0 is relax, 1 is right arm motor imagery, 2 is left arm motor imagery.

if prediction[0] == 0:
    key = Key.up 
    keyboard_game.press(key)
if prediction[0] == 1:
    key = Key.right 
    keyboard_game.press(key)
elif prediction[0] == 2:
    key = Key.left
    keyboard_game.press(key)